In [ ]:
%pip install boto3 litellm aiohttp --quiet --upgrade

<div class="alert alert-block alert-info">
<center>⚠️ <b>Important:</b> Please restart the kernel after installing the dependencies. ⚠️</center>
</div>

# Invoking models from Amazon Bedrock

In [ ]:
import boto3
import json

bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=boto3.Session().region_name
)

def invoke_model(model_id, model_input):
    body = json.dumps(model_input)
    
    response = bedrock.invoke_model(
        modelId=model_id,
        body=body
    )
    
    response_body = json.loads(response['body'].read())
    return response_body

In [5]:
%%time
deepseek_prompt = "What are the key ingredients for pizza?"
deepseek_input = {
    "prompt": f"<｜begin_of_sentence｜><｜User｜>{deepseek_prompt}<｜Assistant｜><think>\n",
    "temperature": 0.5,
    "max_tokens": 512,
}
deepseek_response = invoke_model('us.deepseek.r1-v1:0', deepseek_input)
deepseek_response

CPU times: user 3.99 ms, sys: 2.75 ms, total: 6.74 ms
Wall time: 11.4 s


{'choices': [{'text': 'Okay, the user is asking about the key ingredients for pizza. Let me start by recalling the basic components of a traditional pizza. First, the dough is essential—it\'s made from flour, water, yeast, salt, and maybe a bit of olive oil. Then there\'s the sauce, typically tomato-based with herbs like oregano and basil. Cheese is a big one, usually mozzarella because it melts well. Toppings vary, but classics include pepperoni, mushrooms, bell peppers, onions, and olives. Some people might add meats like sausage or ham, or even go for a white pizza without tomato sauce. Oh, and after baking, maybe a garnish like fresh basil or a drizzle of olive oil. I should also mention that there are variations, like different cheeses or gluten-free crusts for dietary needs. Let me make sure I cover the basics without missing anything important. Maybe list the ingredients in categories: dough, sauce, cheese, toppings, and optional extras. Also, note that some ingredients are opti

In [7]:
%%time
mistral_prompt = "What are the key ingredients for pizza?"
mistral_input = {
    "prompt": f"<s>[INST] {mistral_prompt} [/INST]",
    "temperature": 0.5,
    "max_tokens": 512,
}
mistral_response = invoke_model('mistral.mistral-large-2402-v1:0', mistral_input)
mistral_response

CPU times: user 2.41 ms, sys: 3.3 ms, total: 5.71 ms
Wall time: 4.69 s


{'outputs': [{'text': " The key ingredients for pizza typically include:\n\n1. Pizza Dough: This is the base of the pizza. It's made from flour, yeast, water, salt, and sometimes sugar.\n\n2. Pizza Sauce: This is spread over the dough. It's usually made from tomatoes, but can also be a white sauce or pesto.\n\n3. Cheese: The most common type is mozzarella, but other types like cheddar, gouda, or parmesan can also be used.\n\n4. Toppings: These can vary greatly depending on personal preference. Common toppings include pepperoni, sausage, mushrooms, onions, bell peppers, olives, and tomatoes.\n\n5. Seasonings: These are added to enhance the flavor. They can include basil, oregano, garlic, and red pepper flakes.\n\n6. Olive Oil: Often used to brush the crust for a golden, crispy finish.",
   'stop_reason': 'stop'}]}

## Using the Converse API

In [18]:
%%time
nova_prompt = "What are the key ingredients for pizza?"
messages = [{"role": "user", "content": [{"text":nova_prompt}]}]
nova_response = bedrock.converse(
    modelId="us.amazon.nova-pro-v1:0",
    messages=messages,
    system=[{"text":"You are a helpful assistant."}],
    inferenceConfig={
        "maxTokens": 512,
        "temperature": 0.5,
    }
)
nova_response

CPU times: user 2.45 ms, sys: 4.04 ms, total: 6.48 ms
Wall time: 4.56 s


{'ResponseMetadata': {'RequestId': '51e40bf5-2d23-45e5-bd46-66eb72812c1d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 24 Apr 2025 13:46:03 GMT',
   'content-type': 'application/json',
   'content-length': '1383',
   'connection': 'keep-alive',
   'x-amzn-requestid': '51e40bf5-2d23-45e5-bd46-66eb72812c1d'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'text': 'Certainly! The key ingredients for making a classic pizza typically include:\n\n### Dough:\n1. **Flour** (usually all-purpose or bread flour)\n2. **Yeast** (active dry or instant)\n3. **Water**\n4. **Salt**\n5. **Olive oil** (optional, but commonly used)\n6. **Sugar** (a small amount to feed the yeast, optional)\n\n### Sauce:\n1. **Tomatoes** (canned crushed tomatoes, tomato sauce, or fresh tomatoes)\n2. **Garlic** (minced)\n3. **Olive oil**\n4. **Salt**\n5. **Herbs** (such as oregano, basil, and sometimes thyme or parsley)\n6. **Black pepper**\n\n### Cheese:\n1. **Mozzarella** 

## Using LiteLLM

In [22]:
from litellm import completion

In [25]:
%%time

response = completion(
    model="bedrock/us.amazon.nova-pro-v1:0",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What are the key ingredients for pizza?"}
    ],
    max_tokens=512,
    temperature=0.5,
)
response

CPU times: user 4.57 ms, sys: 3.46 ms, total: 8.02 ms
Wall time: 5.05 s


ModelResponse(id='chatcmpl-2afec966-07fb-4e20-a4bf-d3cbef7bc405', created=1745502573, model='us.amazon.nova-pro-v1:0', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Certainly! The key ingredients for making a classic pizza typically include:\n\n### Dough:\n1. **Flour** (usually all-purpose or bread flour)\n2. **Yeast** (active dry or instant)\n3. **Water** (lukewarm)\n4. **Salt**\n5. **Olive oil** (optional, but commonly used)\n6. **Sugar** (a small amount to feed the yeast, optional)\n\n### Sauce:\n1. **Tomatoes** (canned crushed tomatoes, tomato purée, or fresh tomatoes)\n2. **Garlic** (minced)\n3. **Olive oil**\n4. **Salt**\n5. **Herbs** (such as oregano, basil, and sometimes thyme or parsley)\n6. **Sugar** (a pinch to balance the acidity of the tomatoes, optional)\n\n### Cheese:\n1. **Mozzarella** (fresh or shredded)\n2. **Parmesan** (optional, for added flavor)\n\n### Toppings (can vary widely):\n1. **Pe